# Lecture 1 : Exact Diagonalisation

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PhilipVinc/ComputationalQuantumPhysics/blob/main/Notebooks/1-IntroManyBody/1-basic-study.ipynb)

The goal of this lecture is to get acquainted with the foundational tool of computational quantum mechanics: Exact Diagonalisation (ED). 
ED is a reliable algorithm that produces 'exact' results which can be trusted, at the cost of scaling exponentially.
ED is often used as a starting point and benchmark for approximate, more powerful methods but asymptotically less expensive methods, to check that they indeed give meaningfull results.

The general idea is that ED relies on diagonalising the hamiltonian exactly using some well known diagonalisation routines like [Lanczos](https://en.wikipedia.org/wiki/Lanczos_algorithm), which are implemented in numpy as [`scipy.sparse.linalg.eighs`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigsh.html).
As the computational complexity of diagonalising a $M\times M$ matrix scales roughly with the cube of the matrix size $O(M^3)$, if the size of the hilbert space, and therefore of the matrix grows exponentially with the system size such that $M=2^N$, then the total computational cost is $O(2^{3M})$ so it's exponential.

However, diagonalising the matrix is often not the most expensive part of the algorithm. In the notebook below you will see that the most expensive part is to _construct_ the sparse matrix encoding the Hamiltonian. 
In following notebooks we will therefore investigate on computational techniques to reduce the cost of writing the Hamiltonian.

## General things to keep in mind

When working in computational physics, especially when using Python, there are some rules to keep in mind:

- Keep your code simple. A very messy code can house many bugs, and is hard for anybody else to read. Keep small functions that relate to mathetmatical expressions that you can verify somehow by hand, and then compose them together. Do not write huge functions that do many different things
- Benchmark, benchmark, benchmark: It's important to always know what part of your code is the bottleneck. Often check with `%timeit` or similar techniques what part of your algorithm is the most expensive, and be aware of it!
- **Write vectorized code** : always prefer to use numpy functions like `np.exp(array)` to writing for loops or list comprehensions. Numpy is much faster than for loops! And the code will be easy to run on GPUs one day.
- When testing your code, test it on small systems where the runtime is of the order of few seconds at most. Once you are sure it works, test it on the large systems we are interested in because of the physics.

Also
- Keep in mind that we are physicists, and we want to generate plots in the end. Always think about what quantities we want to plot and aim to compute them!
- When a code takes more than 20 seconds to run, it's usually worth it to 'log' or 'save' many quantities such that you don't have to run it again when you want to plot more stuff. 

### One more thing
- It is a very, very good idea to save to file the data you plot. For example, you can use `np.savez(filename, x_data=x_array, y_data=y_array, ...)` .
    - This will be important, because if at some point you are asked 'consider the data you showed in plot 2, and add this new curve to it' it's much easier if you saved the data to a file

- This is a notebook, but you are not obliged to work on a single notebook. Feel free to create multiple ones, each doing a very specific thing, or to write short scripts.


## Problem Statement:

We will be studying the Transverse Field Ising Model in 1 Dimension

$$
H(J,h) =  J \sum_{\langle i,j\rangle} \sigma_i^z\sigma_j^z -h \sum_i \sigma^x_i 
$$

where J is the ferromagnetic nearest-neighbor coupling, and h is the transverse field.
Remember that by $\sigma_i^x$ we mean

$$
\sigma_i^x = \mathbb{I} \otimes \mathbb{I} \otimes \dots \otimes \sigma^x \otimes \mathbb{I} \dots,
$$
where identities are everywhere except at the site $i$.

Notice that if $h=0$ this is equivalent to the classical ising model.

Without loss of generality we will from now on consider $J=1$ and only vary $h$.

We know that the classical ising Model ($h=0$) has a phase transition if we increase the temperature from a ferromagnetic or antiferromagnetic phase to a disordered phase. 

** How do we identify this phase transition classically? What is the order parameter? **

From now on, we will be looking at the ground state, as well as the first few excited states, defined as

$$
H(h)\ket{\epsilon_i(h)} = \epsilon_i(h) \ket{\epsilon_i(h)}
$$

Notice that the 'classical' term in the hamiltonian, which is $\sum_{\langle i,j\rangle} \sigma_i^z\sigma_j^z$, does not commute with the 'transverse field' term $\sum_i \sigma^x_i $. 

** Can you say something about the ground state of those two different terms, separately? **

### Quantum Phase transition

We want to study what happens if we vary $h$ from 0 to some finite value, like 2.0. We want to look at the magnetization along the z axis defined as

$$
m^z = \frac{1}{N}\sum_i^N \sigma_i^z
$$

You should:
  1. Write the Hamiltonian as a $2^N \times 2^N$ sparse matrix.
      - For this first version, you can first write a set of functions like `sigmax(N: int, i: int) -> scipy.sparse` that take a number of sites and the site index and return the  $2^N \times 2^N$ sparse matrix for this operator.
        - a suggestion: start by writing the $2\times 2$ matrices $\sigma^x$, $\sigma^z$ and identity, then use `scipy.sparse.kron` to take the kronecker product of a list of those operators such that it gives the operators that you want.
      - Then, write an analogous `sigmaz` function and combine them both into a `ising_hamiltonian(N: int, h:float) -> scipy.sparse` function to write the Hamiltonian.

  2. Compute the ground-state energy and wave-function of the hamiltonian by using `scipy.sparse.linalg.eigsh`:
      - Be careful when using `scipy.sparse.linalg.eigsh`: there are two arguments that you must specify which are very important:
        - `k`, the number of eigenvalues/vectors you are interested in. This is 6 by default, and you might see an error if your matrix is smaller than that size. In general, how many eigenvalues do we want? So what could be the value of `k`that makes sense to use?
        - `which`, if you specify that you want only k eigenvalues/vectors, WHICH k should scipy return to you? The largest? the smallest? Be careful in the difference between _algebraic_ magnitude and (absolute) magnitude. We are interested in the former, not the latter!
  
  3. Pick a reasonable system size `N` and a sufficiently dense grid of values for the transverse field `h`. Set `J=1`.
      - Make a plot of the ground-state energy `E_0` (which `eigsh` returns) as a function of `h`.
      - For every value of `h` compute the average magnetization along the `z` axis and make a plot of $\langle m_z\rangle $ as a function of h. 
        - This might look a bit messy for low values of `h`. Do you know why? Think about possible degeneracies.
      - Make the same plot as above, but this time for the squared magnetization $\langle m_z^2\rangle $. What do we learn from it? Is this consistent with the previous plot?

  4. We now want to understand what is the largest system size we could reasonably do, therefore you must benchmark the computational runtime of the `ising_hamiltonian` function for increasing N.
      - Make a plot of the runtime of constructing the Hamiltonian as a function of N. How is this increasing?
      - Compare this cost to diagonalising the matrix with `scipy.sparse.linalg.eigsh`, as a function of N. Which is dominant? What is the largest size you can reasonably simulate on your laptop with this code?





In [ ]:
# If you are running on google colab or on an online jupyter environment and not locally, you might need to install some dependencies
# To run this command, uncomment the line below and execute it.

# ! pip install numpy scipy tqdm quspin

In [1]:
import numpy as np
import scipy as sp
import scipy.sparse as sparse
import matplotlib.pyplot as plt

from tqdm import tqdm

from math import prod

from functools import partial, reduce

In [2]:
# Some useful definitions

_sigmax = np.array([[0,1],[1,0]])
_sigmay = np.array([[0,1j],[-1j,0]])
_sigmaz = np.array([[1,0],[0,-1]])
_id = np.eye(2)


In [ ]:
# Write a sigmax function

def sigmax(N: int, i: int) -> scipy.sparse:
    # ...
    #
    # 

def sigmaz(N: int, i: int) -> scipy.sparse:
    # ....


def ising_hamiltonian(N: int, h: float, J: float = 1) -> scipy.sparse:
    # ...
    

In [ ]:
# If the functions above are well defined, you should be able to do
N = 4
magnetization_z = sum(sigmaz(N, i) for i in range(N)) / N

# and
H_ising = ising_hamiltonian(N=N, h=1.0)

In [ ]:
# For point # 5 - Benchmarking: this is how you can bechmark the performance of a function
import time

n_vals = np.array([2,4,6,8,10,12,14])

# we will repeat the exxecution of the function a few times to lower the variance/noisiness of the measurement.
n_repetitions = 5

times = []
for N in tqdm(n_vals):
    t0 = time.time()
    for _ in range(n_repetitions):
        _ = function_to_benchmark()
    dt = time.time() - t0
    times.append(dt/n_repetitions)

times = np.array(times)